In [44]:

import requests
from bs4 import BeautifulSoup

page = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(page.text, 'html.parser')

records = []

# table recherche
code_table = soup.find('table')

# p items
'''
<p>
    <b>M4A</b><br/>
    <span style="font-size:80%;">
        <a href="/wiki/North_York" title="North York">North York</a>
        <br/>
        (<a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>)
    </span>
</p>,

'''
name_codes_item = code_table.find_all('p')

for mysource in name_codes_item:    
    
    myPostaCode = mysource.find('b').contents[0]
    # <b>M4A</b><br/> -> M4A
    
   
    myBorough_myNeighborhood = mysource.find_all('span')[0].text
    # <a href="/wiki/North_York" title="North York">North York</a> (<a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>)
    
    first_part = myBorough_myNeighborhood.replace( ')', '') 
    first_part1 = first_part.replace( '/', ', ')
    #first_part1
    new_base = first_part1.split('(')
    
    # North York
    myBorough = new_base[0:1]
    myBorough = ','.join(myBorough)
    
    # Victoria Village
    myNeighborhood = new_base[1:]
    myNeighborhood = ','.join(myNeighborhood) 
    if myNeighborhood == '':
        myNeighborhood = myBorough # clone Borough's value if no value to Neighborhood
    else:
        myNeighborhood = myNeighborhood
    
    
    #print( myPostaCode, myBorough,myNeighborhood )
    records.append((myPostaCode,myBorough, myNeighborhood))

import pandas as pd
df_toronto = pd.DataFrame(records, columns=['PostalCode', 'Borough', 'Neighborhood'])

df_toronto.head(10)

import numpy as np
df_toronto.replace("Not assigned", np.nan, inplace=True)
df_toronto.dropna(inplace=True)
df_toronto.reset_index(drop = True, inplace = True)
df_toronto


import numpy as np
df_toronto.replace("Not assigned", np.nan, inplace=True)
df_toronto.dropna(inplace=True)
df_toronto.reset_index(drop = True, inplace = True)

df_toronto

newdf_toronto =df_toronto.replace({
    'Queen\'s Park\n': 'Queen\'s Park',
    'Enclave of M4L': 'East Toronto',
    'Enclave of M5E': 'Downtown Toronto',
    'Downtown TorontoStn A PO Boxes25 The Esplanade': 'Downtown Toronto',
    'Enclave of L4W':'East Toronto',
    'MississaugaCanada Post Gateway Processing Centre': 'Mississauga',
    'East TorontoBusiness reply mail Processing Centre969 Eastern': 'East Toronto'
})
newdf_toronto

len(newdf_toronto)


newdf_toronto.to_csv('toronto_base.csv')

coordinates = pd.read_csv("Geospatial_Coordinates.csv")
coordinates.head()


# rename the column "PostalCode"
coordinates.rename(columns={"Postal Code": "PostalCode"}, inplace=True)
coordinates.head()

# merge two table on the column "PostalCode"
toronto_df_new = newdf_toronto.merge(coordinates, on="PostalCode", how="left")
toronto_df_new.head()

# create a new test dataframe
column_names = ["PostalCode", "Borough", "Neighborhood", "Latitude", "Longitude"]
test_df = pd.DataFrame(columns=column_names)

test_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in test_list:
    test_df = test_df.append(toronto_df_new[toronto_df_new["PostalCode"]==postcode], ignore_index=True)
    
test_df


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
1,M2H,North York,Hillcrest Village,43.803762,-79.363452
2,M4B,East York,"Parkview Hill , Woodbine Gardens",43.706397,-79.309937
3,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
4,M4G,East York,Leaside,43.709060,-79.363452
5,M4M,East Toronto,Studio District,43.659526,-79.340923
6,M1R,Scarborough,"Wexford , Maryvale",43.750072,-79.295849
7,M9V,Etobicoke,"South Steeles , Silverstone , Humbergate , ...",43.739416,-79.588437
8,M9L,North York,Humber Summit,43.756303,-79.565963
9,M5V,Downtown Toronto,"CN Tower , King and Spadina , Railway Lands ...",43.628947,-79.394420


In [45]:
from geopy.geocoders import Nominatim
address = 'Toronto'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [46]:
import folium
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df_new['Latitude'], toronto_df_new['Longitude'], toronto_df_new['Borough'], toronto_df_new['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

In [47]:
# just focus on Scarborough
scarborough_data = test_df[test_df['Borough'] == 'Scarborough'].reset_index(drop=True)
scarborough_data.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
1,M1R,Scarborough,"Wexford , Maryvale",43.750072,-79.295849
2,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353


In [48]:
address = 'Scarborough, Toronto Ontario'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude, longitude))

c:\users\akrithhnayak\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.21.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of Scarborough are 43.773077, -79.257774.


In [49]:
map_scarborough = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(scarborough_data['Latitude'], scarborough_data['Longitude'], scarborough_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_scarborough)  
    
map_scarborough


In [50]:
CLIENT_ID = 'RF2HLDBFJ0Z0IAOHHOTQG4CMUCDM0RGVAF3DVCQFD153FMOB' # your Foursquare ID
CLIENT_SECRET = '2LYBEQRR5TXMPQWUFKMLWCIKWVOA2G3U3Y1YCYL20MWXKRHJ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentials:
CLIENT_ID: RF2HLDBFJ0Z0IAOHHOTQG4CMUCDM0RGVAF3DVCQFD153FMOB
CLIENT_SECRET:2LYBEQRR5TXMPQWUFKMLWCIKWVOA2G3U3Y1YCYL20MWXKRHJ


In [51]:
scarborough_data.loc[0, 'Neighborhood']

'Scarborough Village'

In [52]:
neighborhood_latitude = scarborough_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = scarborough_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = scarborough_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Scarborough Village are 43.7447342, -79.23947609999999.


In [53]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=RF2HLDBFJ0Z0IAOHHOTQG4CMUCDM0RGVAF3DVCQFD153FMOB&client_secret=2LYBEQRR5TXMPQWUFKMLWCIKWVOA2G3U3Y1YCYL20MWXKRHJ&v=20180605&ll=43.7447342,-79.23947609999999&radius=500&limit=100'

In [54]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e7df09f5fb726001c7e42e3'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Eglinton East',
  'headerFullLocation': 'Eglinton East, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 2,
  'suggestedBounds': {'ne': {'lat': 43.749234204500006,
    'lng': -79.23325872538938},
   'sw': {'lat': 43.7402341955, 'lng': -79.2456934746106}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5150a8dae4b045dfb6581c85',
       'name': 'McCowan Park',
       'location': {'lat': 43.74508851212816,
        'lng': -79.239335687338,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.7450885

In [55]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [56]:
from pandas.io.json import json_normalize
import json
# clean the json and structure it into a pandas dataframe.
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

c:\users\akrithhnayak\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


,name,categories,lat,lng
0,McCowan Park,Playground,43.745089,-79.239336
1,Canada Edge Marketers Corporation,Women's Store,43.741680,-79.237060


In [57]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

2 venues were returned by Foursquare.


In [58]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [59]:
# get scarborough venues
scarborough_venues = getNearbyVenues(names=scarborough_data['Neighborhood'],
                                   latitudes=scarborough_data['Latitude'],
                                   longitudes=scarborough_data['Longitude']
                                  )

print(scarborough_venues.shape)
scarborough_venues.head(10)

Scarborough Village
Wexford ,  Maryvale
Malvern ,  Rouge
(10, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Scarborough Village,43.744734,-79.239476,McCowan Park,43.745089,-79.239336,Playground
1,Scarborough Village,43.744734,-79.239476,Canada Edge Marketers Corporation,43.741680,-79.237060,Women's Store
2,"Wexford , Maryvale",43.750072,-79.295849,Crown Pastries,43.746098,-79.293142,Bakery
3,"Wexford , Maryvale",43.750072,-79.295849,Wexford Restaurant,43.746030,-79.293843,Breakfast Spot
4,"Wexford , Maryvale",43.750072,-79.295849,Subway,43.746267,-79.293193,Sandwich Place
5,"Wexford , Maryvale",43.750072,-79.295849,Wexford Heights Plaza,43.746136,-79.293782,Shopping Mall
6,"Wexford , Maryvale",43.750072,-79.295849,Scarborough Garage Door Repair,43.751288,-79.301508,Auto Garage
7,"Wexford , Maryvale",43.750072,-79.295849,Banh Mi Metro,43.745830,-79.295309,Vietnamese Restaurant
8,"Wexford , Maryvale",43.750072,-79.295849,Sequoia Lounge,43.745645,-79.295737,Middle Eastern Restaurant
9,"Malvern , Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant


In [60]:
# find out how many unique categories
scarborough_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Malvern , Rouge",1,1,1,1,1,1
Scarborough Village,2,2,2,2,2,2
"Wexford , Maryvale",7,7,7,7,7,7


In [61]:
print('There are {} uniques categories.'.format(len(scarborough_venues['Venue Category'].unique())))

There are 10 uniques categories.


In [62]:

# analyze each neighborhood

# one hot encoding
scarborough_onehot = pd.get_dummies(scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
scarborough_onehot['Neighbourhood'] = scarborough_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [scarborough_onehot.columns[-1]] + list(scarborough_onehot.columns[:-1])
scarborough_onehot = scarborough_onehot[fixed_columns]

scarborough_onehot.head()

,Neighbourhood,Auto Garage,Bakery,Breakfast Spot,Fast Food Restaurant,Middle Eastern Restaurant,Playground,Sandwich Place,Shopping Mall,Vietnamese Restaurant,Women's Store
0,Scarborough Village,0,0,0,0,0,1,0,0,0,0
1,Scarborough Village,0,0,0,0,0,0,0,0,0,1
2,"Wexford , Maryvale",0,1,0,0,0,0,0,0,0,0
3,"Wexford , Maryvale",0,0,1,0,0,0,0,0,0,0
4,"Wexford , Maryvale",0,0,0,0,0,0,1,0,0,0


In [63]:
scarborough_onehot.shape


(10, 11)

In [73]:
# group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
scarborough_grouped = scarborough_onehot.groupby('Neighbourhood').mean().reset_index()
scarborough_grouped

,Neighbourhood,Auto Garage,Bakery,Breakfast Spot,Fast Food Restaurant,Middle Eastern Restaurant,Playground,Sandwich Place,Shopping Mall,Vietnamese Restaurant,Women's Store
0,"Malvern , Rouge",0.000000,0.000000,0.000000,1.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0
1,Scarborough Village,0.000000,0.000000,0.000000,0.0,0.000000,0.5,0.000000,0.000000,0.000000,0.5
2,"Wexford , Maryvale",0.142857,0.142857,0.142857,0.0,0.142857,0.0,0.142857,0.142857,0.142857,0.0


In [74]:
scarborough_grouped.shape
#scarborough_grouped.head(10)

(3, 11)

In [75]:
num_top_venues = 5

for hood in scarborough_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = scarborough_grouped[scarborough_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Malvern ,  Rouge----
                       venue  freq
0       Fast Food Restaurant   1.0
1                Auto Garage   0.0
2                     Bakery   0.0
3             Breakfast Spot   0.0
4  Middle Eastern Restaurant   0.0


----Scarborough Village----
            venue  freq
0      Playground   0.5
1   Women's Store   0.5
2     Auto Garage   0.0
3          Bakery   0.0
4  Breakfast Spot   0.0


----Wexford ,  Maryvale----
                       venue  freq
0                Auto Garage  0.14
1                     Bakery  0.14
2             Breakfast Spot  0.14
3  Middle Eastern Restaurant  0.14
4             Sandwich Place  0.14




In [76]:
# sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [77]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = scarborough_grouped['Neighbourhood']

for ind in np.arange(scarborough_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scarborough_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Malvern , Rouge",Fast Food Restaurant,Women's Store,Vietnamese Restaurant,Shopping Mall,Sandwich Place,Playground,Middle Eastern Restaurant,Breakfast Spot,Bakery,Auto Garage
1,Scarborough Village,Women's Store,Playground,Vietnamese Restaurant,Shopping Mall,Sandwich Place,Middle Eastern Restaurant,Fast Food Restaurant,Breakfast Spot,Bakery,Auto Garage
2,"Wexford , Maryvale",Vietnamese Restaurant,Shopping Mall,Sandwich Place,Middle Eastern Restaurant,Breakfast Spot,Bakery,Auto Garage,Women's Store,Playground,Fast Food Restaurant


In [81]:
# Run k-means to cluster the neighborhood into 5 clusters.

# set number of clusters
kclusters = 3

scarborough_grouped_clustering = scarborough_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scarborough_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 1, 2])

In [82]:
scarborough_merged = scarborough_data

# add clustering labels
scarborough_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
scarborough_merged = scarborough_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighborhood')

scarborough_merged.head() # check the last columns!


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,0,Women's Store,Playground,Vietnamese Restaurant,Shopping Mall,Sandwich Place,Middle Eastern Restaurant,Fast Food Restaurant,Breakfast Spot,Bakery,Auto Garage
1,M1R,Scarborough,"Wexford , Maryvale",43.750072,-79.295849,1,Vietnamese Restaurant,Shopping Mall,Sandwich Place,Middle Eastern Restaurant,Breakfast Spot,Bakery,Auto Garage,Women's Store,Playground,Fast Food Restaurant
2,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353,2,Fast Food Restaurant,Women's Store,Vietnamese Restaurant,Shopping Mall,Sandwich Place,Playground,Middle Eastern Restaurant,Breakfast Spot,Bakery,Auto Garage


In [85]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(scarborough_merged['Latitude'], scarborough_merged['Longitude'], scarborough_merged['Neighborhood'], scarborough_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [86]:

# Cluster 1 
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 0, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,0,Women's Store,Playground,Vietnamese Restaurant,Shopping Mall,Sandwich Place,Middle Eastern Restaurant,Fast Food Restaurant,Breakfast Spot,Bakery,Auto Garage


In [87]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 1, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,1,Vietnamese Restaurant,Shopping Mall,Sandwich Place,Middle Eastern Restaurant,Breakfast Spot,Bakery,Auto Garage,Women's Store,Playground,Fast Food Restaurant


In [88]:

# Cluster 3
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 2, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Scarborough,2,Fast Food Restaurant,Women's Store,Vietnamese Restaurant,Shopping Mall,Sandwich Place,Playground,Middle Eastern Restaurant,Breakfast Spot,Bakery,Auto Garage
